# II. 영화 리뷰 데이터 전처리(Preprocess)

---
### 1) 데이터 로드 : dataframe (pandas module 사용)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc # garbage collector : 메모리 관리
gc.collect()

In [ ]:
import os

전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'

TRAIN_DATA_FILE = 'ratings_train.txt'
# TEST_DATA_FILE = 'ratings_test.txt'

TRAIN_DATA_PATH = os.path.join(DATA_DIR, TRAIN_DATA_FILE)
# TEST_DATA_PATH = os.path.join(DATA_DIR, TEST_DATA_FILE)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# pandas data frame
train_df = pd.read_table(TRAIN_DATA_PATH) # pd.read_csv(TRAIN_DATA_PATH, sep='\t') 동일한 기능
# test_df = pd.read_table(TEST_DATA_PATH)

---
### 2) 결측 데이터 처리 : nul 데이터는 필요없으므로 먼저 제거한다.

In [ ]:
train_no_nan_df = train_df.dropna(axis=0) # nan이 있는 모든 row를 없애는 코드
# test_no_nan_df = test_df.dropna(axis=0)

---
### 3) 한글 외의 문자는 모두 제거

In [ ]:
# 정규표현식 : Regular Expression
import re

In [ ]:
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

In [ ]:
# 정규 표현식 테스트
txt = '아 더빙.. 진짜 짜증나네요... 목소리 aa'
txt_kor = hangul.sub('', txt) # 한글만 추출
txt_kor

In [ ]:
train_no_nan_df['document_kor'] = train_no_nan_df['document'].apply(lambda x: hangul.sub('', x).strip()).copy() # 빈 칸이 하나 남는 것을 없애기 위해 .strip()을 사용
train_no_nan_df.head()

In [ ]:
train_no_nan_df['document_kor'].replace('', np.nan, inplace=True) # 한글만 남긴 데이터에서 빈 문자열을 NaN값으로 변환한다. 이렇게 하여야 삭제하기 쉽다.

---
한글만 남긴 후 생긴 결측 데이터 확인

In [ ]:
print('null data가 있는가? : ', train_no_nan_df.isnull().values.any())
print('----------')
print('null data 건수 \n', '-----\n', train_no_nan_df.isnull().sum())

In [ ]:
train_no_nan_df.shape

In [ ]:
train_no_nan_df = train_no_nan_df.dropna(axis=0) # nan이 있는 모든 row를 없애는 코드

In [ ]:
train_no_nan_df.shape

In [ ]:
# 149995 - 148739

---
### 4) 중복 데이터 제거

In [ ]:
train_no_nan_df.shape

In [ ]:
train_no_nan_df['document'].nunique()

In [ ]:
train_no_nan_df['document_kor'].nunique()

In [ ]:
145393 - 143461

In [ ]:
# 중복된 데이터 확인
train_no_nan_df[train_no_nan_df.duplicated('document_kor')]

In [ ]:
# train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
dedupe_train_df = train_no_nan_df.drop_duplicates('document_kor', keep='first')

In [ ]:
dedupe_train_df.shape

---
### 5) 형태소 분석
  - 문어체 한국어는 띄어 쓰기에 민감하지 않다.
  - 띄어쓰기만으로 토큰화가 정확하게 되기 어렵다.
  - 문어체 한국어를 토큰화하기 위해서는 형태소 분석이 필요하다.

In [ ]:
# !pip install kiwipiepy

In [ ]:
# 데이터 확인
dedupe_train_df['document_kor']

---
kiwi를 이용한 한국어 형태소 분석  
https://github.com/bab2min/kiwipiepy

In [ ]:
# 한국어 형태소 분석을 위한 모듈 임포트
from kiwipiepy import Kiwi

In [ ]:
kiwi = Kiwi()
kiwi.prepare()

---
형태소 분석 결과 참고 : 품사 태그  
https://github.com/bab2min/kiwipiepy#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8  

In [ ]:
# kiwi 형태소 분석 예
kiwi.analyze(txt_kor)

In [ ]:
 dedupe_train_df['document_kor']

In [ ]:
import time

In [ ]:
start = time.time() # start time

dedupe_train_df['morphed'] = dedupe_train_df['document_kor'].apply(lambda x: kiwi.analyze(x)[0]) # 2 분 정도 걸림. # data 중에 형식이 맞지 않는 데이터가 입력되면 에러가 난다. NaN이 있는 경우
# df_new = dedupe_kor_df.apply(lambda x: (kiwi.analyze(x))[0])

end = time.time() # end time

print("time : ", end - start, " sec") # sec

In [ ]:
dedupe_train_df

In [ ]:
# kiwi의 형태소 분석 결과 중 단어만을 선택하기 위한 함수
def get_morphed_word(wiki_result):
    txt = []
    for i in wiki_result[0]:
        txt.append(i[0])
    # print(txt)
    
    return txt

In [ ]:
morphed_txt = get_morphed_word(dedupe_train_df['morphed'][0])
morphed_txt

In [ ]:
dedupe_train_df['morphed_word'] = dedupe_train_df['morphed'].apply(lambda x: get_morphed_word(x))

In [ ]:
dedupe_train_df.loc[200:220, 'morphed_word']

In [ ]:
stopwords=['가', '걍', '과', 'ᆫ', 'ᆫ가', 'ᆫ다', 'ᆫ지', '는', '다', '도', '들', '등', 'ᆯ', '를', '보', '수', '아', '었', '에', '에서', '와', '으로', '은', '을', '의', '이', '있'
           '자', '잘', '좀', '하', '하다','한']

In [ ]:
morphed_txt_2 = dedupe_train_df['morphed_word'][219]
morphed_txt_2

In [ ]:
[w for w in morphed_txt_2 if not w in stopwords]

In [ ]:
dedupe_train_df['no_stopwords'] = dedupe_train_df['morphed_word'].apply(lambda x: [w for w in x if not w in stopwords])

In [ ]:
type(dedupe_train_df['no_stopwords'][0])

In [ ]:
dedupe_train_df['no_stopwords'][0]

---
### 6) 토큰화(Tokenization)

In [ ]:
train_x = dedupe_train_df['no_stopwords'] # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
# train_x = dedupe_train_df['no_stopwords'].apply(lambda x: ' '.join(x)) # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
train_y = dedupe_train_df['label']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import sequence

---
(가) 단어 사전을 만들기 위해 첫 번째 토큰화 작업
  - 디폴트 값으로 토큰화하여 결과를 관찰하고 참고 한다.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x)  # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합(vocabulary)을 생성한다.

In [ ]:
# 분석한 결과를 보기 : 전체 어휘의 숫자와 희귀단어 분석

threshold = 3 # 등장 빈도 하한, 이 수보다 작은 빈도를 나타내는 어휘는 희귀 단어로 정한다.
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# print(tokenizer.word_index)
# print(tokenizer.word_counts)

In [ ]:
# 전체 어휘 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

---
(나) 앞에서 만든 단어 사전을 이용하여 새롭게 토큰화
  - Tokenizer()를 초기화하고 다시 토큰화 한다.

In [ ]:
# vocab_size = 17199

oov_tok = "<OOV>" # Out-Of-Vocabulary 토큰

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_x)

  - 설정값을 바꾸고 다시 토큰화해도 전체 어휘의 수는 변하지 않는다. 인코딩할 때 사용할 어휘 기준만 달라진다.

In [ ]:
len(tokenizer.word_counts) # 사용된 전체 어휘의 수, 토큰의 수
# print(tokenizer.word_index)

In [ ]:
# type(tokenizer.word_counts) # collections.OrderedDict : 입력된 순서대로 값을 가지고 있는 딕셔너리

In [ ]:
# tokenizer.word_counts['영화'] # '영화'란 어휘가 몇 번 사용되었는지 출력
# sorted(tokenizer.word_counts.items()) # 키 값으로 정렬

In [ ]:
# 밸류로 정렬, 사용 빈도가 많은 어휘부터 볼 수 있다.
import operator

sorted_dict = sorted(tokenizer.word_counts.items(), key=operator.itemgetter(1), reverse=True)
sorted_dict

In [ ]:
tokenizer.word_index # 토큰화 될 때, 사용 빈도에 따라 인덱스 값이 주어진 것을 확인. 

---
(다) 토큰화 정보를 파일로 저장한다.
  - 이렇게 저장하면 필요할 때 읽어서 사용할 수 있다. 특히 추론할 때 잘 사용할 수 있다.

In [ ]:
# tokenizer.get_config() # 설정값과 토큰화 한 정보를 볼 수 있다.

# 토큰화 정보를 json 형식으로 파일에 저장
# DATA_DIR = 'data'
TOKENIZED_FILE = 'tokenized.json'
TOKENIZED_PATH = os.path.join(DATA_DIR, TOKENIZED_FILE)

import json

tok_json = tokenizer.to_json()
# with io.open(DATA_IN_DIR + 'tokenizer_' + ver + '.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(tokenizer_json, ensure_ascii=False))

json.dump(tok_json, open(TOKENIZED_PATH, 'w'), ensure_ascii=False)

# 읽는 방법 : 아래 2 가지 방법 중 한 가지 방법으로 읽어 올 수 있다.
'''
tok_configs = None
with open(tokenized_file, 'r') as f:
    tok_configs = json.load(f)
'''
# keras.preprocessing.text.tokenizer_from_json(json_string)
'''
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
'''

---
### 7) 시퀀싱과 패딩(Sequencing and Padding)  
토큰화 된 데이터를 숫자(인덱스) 시퀀스로 바꾸고 같은 길이의 시퀀스로 만들기 위해 패딩을 한다.

In [ ]:
# 문자열 리스트로 이루어진 입력데이터를 숫자 시퀀스로 변경한다. 
training_sequences = tokenizer.texts_to_sequences(train_x)

In [ ]:
# training_sequences
training_sequences[0]

In [ ]:
text = tokenizer.sequences_to_texts([training_sequences[0]]) # 역으로 문장 확인
text

In [ ]:
# Defining pre-processing hyperparameters
max_len = 30 # 시퀀스의 길이를 30으로 고정
trunc_type = "post" # 길이가 30 보다 길 때 뒷 부분을 버린다. 
padding_type = "post" # 길이가 30 보다 짧을 대 뒷 부분을 0으로 채운다.

# padding : 뒤를 0으로 채운다.
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [ ]:
training_sequences[0] # 확인

In [ ]:
train_padded[0]

In [ ]:
text_paddes = tokenizer.sequences_to_texts([train_padded[0]]) # 역으로 문장 확인
text_paddes

In [ ]:
# Shape of train tensor
print('Shape of training tensor: ', train_padded.shape)

In [ ]:
# Before padding
len(training_sequences[0]), len(training_sequences[1])

In [ ]:
# After padding
len(train_padded[0]), len(train_padded[1])

---
### 8) 전처리(PreProcessed)된 Data를 파일에 저장

dataframe의 객체를 그대로 저장하려면, (list, dict 등 포함) pickle로 저장한다.    
to_pickle, read_pickle도 사용 가능 :   
https://wikidocs.net/8929  
https://tariat.tistory.com/739  

In [ ]:
# DATA_DIR = 'data'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

PROCESSED_DATA_FILE = "train_padded.p"
PROCESSED_LABEL_FILE = "train_label.p"

PROCESSED_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_DATA_FILE)
PROCESSED_LABEL_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_LABEL_FILE)

In [ ]:
train_y

In [ ]:
# 데이터 저장

import pickle

with open(PROCESSED_DATA_PATH, "wb") as file:
    pickle.dump(train_padded, file)
    
with open(PROCESSED_LABEL_PATH, "wb") as file:
    pickle.dump(train_y, file)

In [ ]:
# 저장된 파일 로드 확인
with open(PROCESSED_DATA_PATH, "rb" ) as file:
    train_padded_loaded = pickle.load(file)

In [ ]:
train_padded_loaded

---
### 9) Test dataset 전처리

트레이닝 후 정확도를 측정하기 위해 사용하기 위해 테스트 데이터셋도 전처리해 둔다.
  1. test dataset load 
  2. 결측 데이터 제거
  3. 한글만 추출
  4. 형태소 분석
  5. 토큰화
  6. 시퀀스 만들기와 패딩

In [ ]:
TEST_DATA_FILE = 'ratings_test.txt'
TEST_DATA_PATH = os.path.join(DATA_DIR, TEST_DATA_FILE)

In [ ]:
test_df = pd.read_table(TEST_DATA_PATH)

In [ ]:
test_no_nan_df = test_df.dropna(axis=0) # 결측 데이터 제거
# test_no_nan_df

In [ ]:
test_no_nan_df['document_kor'] = test_no_nan_df['document'].apply(lambda x: hangul.sub('', x).strip()).copy() # 빈 칸이 하나 남는 것을 없애기 위해 .strip()을 사용
test_no_nan_df.head()

In [ ]:
test_no_nan_df['document_kor'].replace('', np.nan, inplace=True) # 한글만 남긴 데이터에서 빈 문자열을 NaN값으로 변환한다. 이렇게 하여야 삭제하기 쉽다.

In [ ]:
print('null data가 있는가? : ', test_no_nan_df.isnull().values.any())
print('----------')
print('null data 건수 \n', '-----\n', test_no_nan_df.isnull().sum())

In [ ]:
test_no_nan_df.shape

In [ ]:
test_no_nan_df = test_no_nan_df.dropna(axis=0) # nan이 있는 모든 row를 없애는 코드

In [ ]:
test_no_nan_df.shape

In [ ]:
test_no_nan_df['document_kor'].nunique()

In [ ]:
# 중복된 데이터 확인
test_no_nan_df[test_no_nan_df.duplicated('document_kor')]

In [ ]:
# document_kor 열에서 중복인 내용이 있다면 중복 제거
test_no_nan_df = test_no_nan_df.drop_duplicates('document_kor', keep='first')

In [ ]:
test_no_nan_df.shape

In [ ]:
start = time.time() # start time

test_no_nan_df['morphed'] = test_no_nan_df['document_kor'].apply(lambda x: kiwi.analyze(x)[0]) # 40 분 정도 걸림. # data 중에 형식이 맞지 않는 데이터가 입력되면 에러가 난다. NaN이 있는 경우
# df_new = dedupe_kor_df.apply(lambda x: (kiwi.analyze(x))[0])

end = time.time() # end time

print("time : ", end - start, " sec") # sec

In [ ]:
test_no_nan_df

In [ ]:
test_no_nan_df['morphed_word'] = test_no_nan_df['morphed'].apply(lambda x: get_morphed_word(x))

In [ ]:
test_no_nan_df

In [ ]:
test_x = test_no_nan_df['morphed_word'] # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
# train_x = dedupe_train_df['no_stopwords'].apply(lambda x: ' '.join(x)) # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
test_y = test_no_nan_df['label']

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_x)

In [ ]:
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [ ]:
# Shape of test tensor
print('Shape of test tensor: ', test_padded.shape)

# testing_sequences = tokenizer.texts_to_sequences(test_x)
# testing_padded = pad_sequences(testing_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

In [ ]:
PROCESSED_TEST_DATA_FILE = "test_padded.p"
PROCESSED_TEST_LABEL_FILE = "test_label.p"

PROCESSED_TEST_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_TEST_DATA_FILE)
PROCESSED_TEST_LABEL_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_TEST_LABEL_FILE)

In [ ]:
# 데이터 저장
with open(PROCESSED_TEST_DATA_PATH, "wb") as file:
    pickle.dump(test_padded, file)
    
with open(PROCESSED_TEST_LABEL_PATH, "wb") as file:
    pickle.dump(test_y, file)